In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


<img src="./images/multiLayer perceptron1.png" align="left" width="1500"/>

<img src="./images/multiLayer perceptron2.png" align="left" width="1500"/>

<img src="./images/multiLayer perceptron3.png" align="left" width="1500"/>

<img src="./images/multiLayer perceptron4.png" align="left" width="1500"/>

<img src="./images/multiLayer perceptron5.png" align="left" width="1000"/>

XOR 연산은 입력 데이터를 X라고 할 경우[[0,0], [0, 1], [1, 0], [1, 1]]과 같은 경우의 수가 있다.  
총 4개의 데이터가 각각 2개의 특성을 가지고 있으므로 X는 [4, 2]의 형태로 정의할 수 있다.  
입력값에 따른 출력값 Y는 [[0], [1], [1], [0]]이므로 Y는 [4, 1]의 형태로 정의할 수 있다.

In [2]:
X = tf.placeholder(dtype=tf.float32, shape=[4, 2]) # 입력값
Y = tf.placeholder(dtype=tf.float32, shape=[4, 1]) # 출력값

1번째 히든 레이어에서 2개의 입력(X)과 1개의 편향값(B1)을 받아서 2개의 시그모이드 출력(Z)을 다음 레이어로 전달한다.

In [3]:
# 2개의 입력을 받는 2개의 뉴런을 만든다.
# random_normal() 메소드는 정규 분포로 부터 난수값을 반환하고 random_uniform()은 균등 분포로 부터 난수값을 반환한다.
W1 = tf.Variable(tf.random_normal([2, 2]))
# 각 뉴런은 1개의 편향값을 가진다.
# zeros() 메소드는 모든 원소의 값이 0인 텐서(2차원 이상의 행렬)를 생성한다.
B1 = tf.Variable(tf.zeros([2]))
# 시그모이드 함수를 거쳐 출력값으로 Z를 리턴한다. => tf.sigmoid(W1 * X + B1)
Z = tf.sigmoid(tf.matmul(X, W1) + B1)

In [4]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print(sess.run(W1)) # 랜덤값
print(sess.run(B1))
train_X = [[0,0], [0, 1], [1, 0], [1, 1]]
print(sess.run(Z, feed_dict={X: train_X}))
sess.close()

[[ 0.8811427   0.953503  ]
 [-0.64716905 -1.4921741 ]]
[0. 0.]
[[0.5        0.5       ]
 [0.34362775 0.18359566]
 [0.7070589  0.7218191 ]
 [0.558228   0.36849678]]


2번째 히든 레이어에서 1번째 히든 레이어의 출력값인 Z와 1개의 편향값을 받아서 1개의 시그모이드를 출력(Y_hat)한다.

In [5]:
# Z를 입력값으로 받는 1개의 뉴런을 만든다.
W2 = tf.Variable(tf.random_normal([2, 1]))
# 각 뉴런은 1개의 편향값을 가진다.
B2 = tf.Variable(tf.zeros([1]))
# 시그모이드 함수를 거쳐 출력값으로 Z를 리턴한다. => tf.sigmoid(W2 * X + B2)
Y_hat = tf.sigmoid(tf.matmul(Z, W2) + B2)

In [6]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print(sess.run(W2))
print(sess.run(B2))
print(sess.run(Y_hat, feed_dict={X: train_X}))
sess.close()

[[-1.2838403]
 [ 1.0382979]]
[0.]
[[0.4693457 ]
 [0.42183816]
 [0.6240465 ]
 [0.53369373]]


손실 함수로 크로스 엔트로피를 사용하고, 경사 하강법으로 모델의 매개 변수(가중치, 편향)을 최적화 한다.

In [7]:
# 크로스 엔트로피
loss = tf.reduce_mean(
    -1 * (Y * tf.log(Y_hat) + (1.0 - Y) * tf.log(1.0 - Y_hat))
)
# 경사 하강법
train_step = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
# 모델을 학습하기 위한 데이터를 만든다.
train_X = [[0,0], [0, 1], [1, 0], [1, 1]]
train_Y = [[0], [1], [1], [0]]

학습을 진행한다.

In [8]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print('학습 데이터: {}'.format(train_X))
    print('=' * 80)
    
    for epoch in range(20001):
        sess.run(train_step, feed_dict={X: train_X, Y: train_Y})
        if epoch % 5000 == 0:
            print('epoch: {}'.format(epoch))
            print('output\n', sess.run(Y_hat, feed_dict={X: train_X, Y: train_Y}), sep='')
            print('=' * 80)

학습 데이터: [[0, 0], [0, 1], [1, 0], [1, 1]]
epoch: 0
output
[[0.49093974]
 [0.485585  ]
 [0.47344896]
 [0.47223377]]
epoch: 5000
output
[[0.1777032 ]
 [0.7199008 ]
 [0.5523686 ]
 [0.60113823]]
epoch: 10000
output
[[0.04285073]
 [0.9442527 ]
 [0.5024451 ]
 [0.5256334 ]]
epoch: 15000
output
[[0.0204283 ]
 [0.9771537 ]
 [0.4989364 ]
 [0.51088995]]
epoch: 20000
output
[[0.01316363]
 [0.98650277]
 [0.49867392]
 [0.50651467]]


학습 결과를 통해 [0, 0], [1, 1]은 0에 상당히 가까운 값을 출력하고 [0, 1], [1, 0]은 1에 상당히 가까운 값을 출력하는 것을 확인할 수 있다.